#### 当模型训练和推理时, 数据应该在GPU上
#### 当使用后处理(**可视化或保存结果**)时, 数据需要转回CPU上
#### 某些库(numpy, pandas, openCV等)只能在CPU上运行, 使用时必须将数据转到CPU上 

```
# 训练阶段
inputs = load_data().to('cuda')  # CPU→GPU
outputs = model(inputs)
loss = loss_fn(outputs, labels.to('cuda'))

# 日志记录
if step % 100 == 0:
    print(f"Loss: {loss.detach().cpu().item()}")  # GPU→CPU

# 结果保存
final_result = postprocess(outputs.detach().cpu())  # 后处理需CPU
torch.save(final_result, 'result.pt')
```

在 PyTorch 中，detach() 方法返回一个新的 Tensor，这个 Tensor 和原来的 Tensor 共享相同的内存空间，但是不会被计算图所追踪，也就是说它不会参与反向传播，不会影响到原有的计算图, 通常在以下两种情况下使用：
1. 在计算图中间，需要截断反向传播的梯度计算时。例如，当计算某个 Tensor 的梯度时，我们希望在此处截断反向传播，而不是将梯度一直传递到计算图的顶部，从而减少计算量和内存占用。
2. 在将 Tensor 从 GPU 上拷贝到 CPU 上时，由于 Tensor 默认是在 GPU 上存储的，所以直接进行拷贝可能会导致内存不一致的问题。此时可以使用 detach() 方法先将 Tensor 分离出来，然后再将分离出来的 Tensor 拷贝到 CPU 上。

In [ ]:
import torch

# 危险操作（可能导致梯度错误）
a = torch.tensor([2.0], requires_grad=True)
b = a.data
b += 1  # 修改原始数据但无报警
loss = (a * 3).sum()
loss.backward()  # 梯度计算基于被篡改的值

# 安全操作
a = torch.tensor([2.0], requires_grad=True)
b = a.detach()
b += 1  # 触发 RuntimeError: a leaf Variable that requires grad...

In [3]:
import torch

a = torch.tensor([1.0], requires_grad=True)
# b和a共享同一块内存
b = a.detach()
b[0] = 5.0
print(a)  # 输出 tensor([5.], requires_grad=True)

tensor([5.], requires_grad=True)
